<a href="https://colab.research.google.com/github/DJCordhose/ai/blob/master/notebooks/process/export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline and Export

* https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
* https://scikit-learn.org/stable/modules/model_persistence.html

In [1]:
!rm -rf data && mkdir data && cd data && curl https://raw.githubusercontent.com/DJCordhose/deep-learning-crash-course-notebooks/master/data/insurance-customers-1500.csv -O && cd -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 26783  100 26783    0     0   502k      0 --:--:-- --:--:-- --:--:--  502k
/content


In [2]:
!ls -l data

total 28
-rw-r--r-- 1 root root 26783 Jul 25 10:42 insurance-customers-1500.csv


In [0]:
import pandas as pd
df = pd.read_csv('data/insurance-customers-1500.csv', sep=';')

In [4]:
stats = df.describe()
stats

,speed,age,miles,group
count,1500.000000,1500.000000,1500.000000,1500.000000
mean,122.492667,44.980667,30.434000,0.998667
std,17.604333,17.130400,15.250815,0.816768
min,68.000000,16.000000,1.000000,0.000000
25%,108.000000,32.000000,18.000000,0.000000
50%,120.000000,42.000000,29.000000,1.000000
75%,137.000000,55.000000,42.000000,2.000000
max,166.000000,100.000000,84.000000,2.000000


In [0]:
y=df['group']

In [0]:
df.drop('group', axis='columns', inplace=True)

In [0]:
X = df.values

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1200, 3), (1200,), (300, 3), (300,))

In [9]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=13, min_samples_leaf=1, min_samples_split=8, n_estimators=14, n_jobs=-1)
%time clf.fit(X_train, y_train)

CPU times: user 29.6 ms, sys: 6.12 ms, total: 35.7 ms
Wall time: 110 ms


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=13, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=14, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [10]:
train_score = clf.score(X_train, y_train)
train_score

0.9108333333333334

In [11]:
test_score = clf.score(X_test, y_test)
test_score

0.7666666666666667

In [12]:
from sklearn.model_selection import cross_val_score
cross_val_scores = cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=10)
cross_val_scores

array([0.72727273, 0.80833333, 0.79166667, 0.78333333, 0.71666667,
       0.76666667, 0.85833333, 0.775     , 0.79166667, 0.80672269])

In [13]:
# mean score and the 95% confidence interval (2 standard deviations assuming normal distribution)
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_val_scores.mean(), cross_val_scores.std() * 2))

Accuracy: 0.78 (+/- 0.08)


In [14]:
scores = {
    'cross_val_scores': cross_val_scores,
    'train_score': train_score,
    'test_score': test_score
}
scores

{'cross_val_scores': array([0.72727273, 0.80833333, 0.79166667, 0.78333333, 0.71666667,
        0.76666667, 0.85833333, 0.775     , 0.79166667, 0.80672269]),
 'test_score': 0.7666666666666667,
 'train_score': 0.9108333333333334}

In [0]:
!rm -rf models && mkdir models

In [0]:
import pickle

pickle.dump(clf, open('models/model.pickle', 'wb'))

In [0]:
!rm -rf stats && mkdir stats

In [0]:
# has the distribution of the data changed substantially?
pickle.dump(stats, open('stats/describe.pickle', 'wb'))

In [0]:
# has the score descread since last time or is suspiciously different?
pickle.dump(scores, open('stats/scores.pickle', 'wb'))

In [21]:
import numpy, sklearn, pandas

versions = {
    'numpy': numpy.__version__,
    'sklearn': sklearn.__version__, 
    'pandas': pandas.__version__,
    'model': 1,
    'model_name': 'plain_rf'

}
versions

{'model': 1,
 'model_name': 'plain_rf',
 'numpy': '1.16.4',
 'pandas': '0.24.2',
 'sklearn': '0.21.2'}

In [0]:
# we need to have exactly the same versions for serving
pickle.dump(versions, open('stats/versions.pickle', 'wb'))

In [23]:
!ls -l stats

total 12
-rw-r--r-- 1 root root 1145 Jul 25 10:43 describe.pickle
-rw-r--r-- 1 root root  372 Jul 25 10:43 scores.pickle
-rw-r--r-- 1 root root  132 Jul 25 10:43 versions.pickle


In [24]:
!ls -l

total 136
drwxr-xr-x 2 root root  4096 Jul 25 10:42 data
-rw-r--r-- 1 root root 26688 Jul 25 10:29 export.ipynb
-rw-r--r-- 1 root root    15 Jul 25 10:43 export-pipeline.ipynb
drwxr-xr-x 2 root root  4096 Jul 25 10:42 models
-rw-r--r-- 1 root root 89706 Jul 25 10:35 prod.tgz
drwxr-xr-x 1 root root  4096 Jul 19 16:14 sample_data
drwxr-xr-x 2 root root  4096 Jul 25 10:43 stats


In [25]:
# we also need the code that created this to have it reproduceable
!cd models
!curl https://raw.githubusercontent.com/DJCordhose/ml-workshop/master/notebooks/process/export.ipynb -O
!cd -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 26688  100 26688    0     0   120k      0 --:--:-- --:--:-- --:--:--  120k
/


In [26]:
!ls -l models

total 68
-rw-r--r-- 1 root root 66179 Jul 25 10:42 model.pickle


In [0]:
# have everything we need to reproduce and compare to other versions here
# better to version this 
# often versioning data is a challenge simply because of its size
!tar -czf prod.tgz models stats data

In [28]:
!ls -l

total 76
drwxr-xr-x 2 root root  4096 Jul 25 10:42 data
-rw-r--r-- 1 root root 26688 Jul 25 10:43 export.ipynb
-rw-r--r-- 1 root root    15 Jul 25 10:43 export-pipeline.ipynb
drwxr-xr-x 2 root root  4096 Jul 25 10:42 models
-rw-r--r-- 1 root root 26565 Jul 25 10:43 prod.tgz
drwxr-xr-x 1 root root  4096 Jul 19 16:14 sample_data
drwxr-xr-x 2 root root  4096 Jul 25 10:43 stats


In [29]:
!date

Thu Jul 25 10:43:21 UTC 2019


# Exercise: Export your favorite model

* choose any strategy you like
* save all the data you need for serving
* can you come up with some automated safety check for a new model?
* should you deploy to production automatically anyway?